In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
from IPython.core.display import display, HTML
from IPython.display import clear_output

In [2]:
keywords=['Method', 'pH', 'Temperature', 'K', 'SD', 'logK', 'Charge', 'N', 'Year', 'Journal', 'IUPAC', 'Name', 'Remarks']
types={'charge':float, 'iupac':str, 'journal':str, 'k':float, 'logk':float, 'method':str, 'sd':float, 'n':float, 
       'name':str, 'ph':float,  'temperature':float, 'year':float, 'remarks':str}
mappings = {'K' : 'K (experimental)',
           'SD' : 'SD of K',
           'IUPAC' : 'IUPAC Name'}

In [3]:
login_url = 'http://interactions.cyclodextrin.net/web/login'

payload = {
    "username": "cycloguest", 
    "pwd": "cycloguest"
}

In [4]:
session_requests = requests.session()
result = session_requests.post(login_url,  data = payload,  headers = dict(referer=login_url))

In [8]:
#BeautifulSoup(result.text, "html.parser")

In [9]:
url = r'http://interactions.cyclodextrin.net/search/info'

my_cookies = [{ 'name':'sectionPage', 'value': 'Home'},
              {'name':'searchPage', 'value': 'quick'},
              {'name' : 'resultPage', 'value':'inters'},
              {'name' : 'browseQuery', 'value':'result'},
              {'name':'infoId', 'value': str(0)},
              {'name':'infoPage', 'value':'inter'}]

def scarpePage(infoPage, infoId, keywords=keywords, mappings=mappings):
    record={}
    my_cookies[-1]={'name':'infoPage', 'value':infoPage}
    my_cookies[-2]={'name':'infoId', 'value': str(infoId)}
    for cookie in my_cookies:
        session_requests.cookies.set(**cookie, domain='.interactions.cyclodextrin.net')

    response = session_requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")


    for row in soup.find_all("tr"):
        if '<span id="main_text_subsection">' in str(row):
            record['id'] = row.find_all('span')[-1].contents[0].replace('[id', '').replace(']', '')
            continue
        for keyword in keywords:
            if keyword in mappings.keys():
                keyword_m=mappings[keyword]
            else:
                keyword_m=keyword
                
            if len(row.find_all('span', attrs={'id':'main_text_body_small'}, text=keyword_m)):
                try:
                    record[keyword.lower()] = types[keyword.lower()](row.find_all('span')[-1].contents[0])
                except:
                    record[keyword.lower()] = str(row.find_all('span')[-1].contents[0])
                continue
    return record, response

In [11]:
scarped

[{'ligand_id': '', 'cyclo_id': '', 'inter_id': ''}]

In [16]:
scarped = []
for infoId in range(15000):
    record={}
    for infoPage in ['ligand', 'cyclo', 'inter']:
        rec, _ = scarpePage(infoPage, infoId)
        rec = {('%s_%s'%(infoPage,k)):v for k,v in rec.items()}
        record.update(**rec)
    if len(record):
        record['id']=record['ligand_id']
        for k in list(record.keys()):
            if '_id' in k:
                del record[k]
        scarped.append(record)
        
    clear_output()
    scarped_df=pd.DataFrame.from_records(scarped, index='id').dropna(how='all')
    display(scarped_df)
    display(pd.DataFrame(scarped_df.dtypes).T)

,cyclo_name,inter_charge,inter_journal,inter_k,inter_logk,inter_method,inter_n,inter_ph,inter_remarks,inter_sd,inter_temperature,inter_year,ligand_iupac,ligand_name
id,,,,,,,,,,,,,,
1,α-cyclodextrin,NaN,"Recueil, 1970 (89), 845",4.000000e-01,-0.39794,Spectrophotometry,1.0,12.0,NaN,0.032,25.0,1970.0,4-nitrophenol,"""isopropyl p-nitrophenyl methylphosphonate; R(..."
2,α-cyclodextrin,NaN,"Recueil, 1970 (89), 845",3.400000e-01,-0.46852,Spectrophotometry,1.0,12.3,NaN,0.050,25.0,1970.0,4-nitrophenol,"""isopropyl p-nitrophenyl methylphosphonate; R(..."
3,α-cyclodextrin,NaN,"Recueil, 1970 (89), 845",3.100000e-01,-0.50864,Spectrophotometry,1.0,12.6,NaN,0.080,25.0,1970.0,4-nitrophenol,"""isopropyl p-nitrophenyl methylphosphonate; R(..."
4,α-cyclodextrin,NaN,"Recueil, 1970 (89), 845",2.900000e-01,-0.53760,Spectrophotometry,1.0,12.8,NaN,0.020,25.0,1970.0,4-nitrophenol,"""isopropyl p-nitrophenyl methylphosphonate; R(..."
5,α-cyclodextrin,NaN,"Recueil, 1970 (89), 845",2.600000e-01,-0.58503,Spectrophotometry,1.0,12.9,NaN,0.140,25.0,1970.0,4-nitrophenol,"""isopropyl p-nitrophenyl methylphosphonate; R(..."
6,α-cyclodextrin,NaN,"Recueil, 1970 (89), 845",7.700000e-01,-0.11351,Spectrophotometry,1.0,NaN,in acid medium,0.180,NaN,1970.0,4-[(E)-2-[4-(dimethylamino)phenyl]diazen-1-yl]...,methyl orange(racemic mixture)
7,α-cyclodextrin,NaN,"Recueil, 1970 (89), 845",8.000000e-02,-1.09691,Spectrophotometry,1.0,NaN,in acid medium,NaN,NaN,1970.0,4-[(E)-2-[4-(dimethylamino)phenyl]diazen-1-yl]...,methyl orange(racemic mixture)
8,α-cyclodextrin,NaN,"J. Pharm. Sci., 65(3), 379-83",2.000000e+02,2.30103,Potentiometric titration,1.0,NaN,NaN,NaN,25.0,1976.0,4-nitrophenol,p-nitrophenol
9,β-cyclodextrin,NaN,"J. Pharm. Sci., 65(3), 379-83",2.200000e+03,3.34242,Potentiometric titration,1.0,NaN,NaN,NaN,25.0,1976.0,4-nitrophenol,p-nitrophenol


,cyclo_name,inter_charge,inter_journal,inter_k,inter_logk,inter_method,inter_n,inter_ph,inter_remarks,inter_sd,inter_temperature,inter_year,ligand_iupac,ligand_name
0,object,object,object,float64,float64,object,float64,float64,object,float64,float64,float64,object,object


In [ ]:
scarped_df=pd.DataFrame.from_records(scarped, index='id').dropna(how='all')

In [ ]:
scarped_df.to_pickle('data.pkl')
        

In [ ]:
_, resp = scarpePage('inter', 13153, keywords=keywords, mappings=mappings)

In [ ]:
HTML(str(BeautifulSoup(resp.text, "html.parser")))